In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install earthengine-api --upgrade
import ee
ee.Authenticate()
ee.Initialize(project='**')

import pandas as pd
import time
import os

BASE_PATH = '**'
ATTRIBUTE_NAME = '**'

FILE_NAME = '**'
INPUT_CSV_PATH = '**'
OUTPUT_CSV_PATH = '**'

YEAR_COLUMN_NAME = 'Year'
LAT_COLUMN_NAME = 'lat'
LON_COLUMN_NAME = 'lon'

SAVE_INTERVAL = 1000

daegu_2024 = ee.FeatureCollection(BASE_PATH + 'forest_daegu_2024')
gyeongbuk_2024_p1 = ee.FeatureCollection(BASE_PATH + 'forest_gyeongbuk_2024_part1')
gyeongbuk_2024_p2 = ee.FeatureCollection(BASE_PATH + 'forest_gyeongbuk_2024_part2')
forest_map_2024 = daegu_2024.merge(gyeongbuk_2024_p1).merge(gyeongbuk_2024_p2)

daegu_2019 = ee.FeatureCollection(BASE_PATH + 'forest_daegu_2019')
gyeongbuk_2019 = ee.FeatureCollection(BASE_PATH + 'forest_gyeongbuk_2019')
forest_map_2019 = daegu_2019.merge(gyeongbuk_2019)

forest_map_2022 = ee.FeatureCollection(BASE_PATH + 'forest_2022_merged')

def get_forest_info_for_year(year, lon, lat):
    try:
        if year == 2024:
            selected_map = forest_map_2024
        elif 2019 <= year <= 2021:
            selected_map = forest_map_2019
        elif 2022 <= year <= 2023:
            selected_map = forest_map_2022
        else:
            return '연도 범위 없음'

        point = ee.Geometry.Point([lon, lat])
        filtered = selected_map.filterBounds(point)

        if filtered.size().getInfo() == 0:
            return -1

        feature = ee.Feature(filtered.first())
        code = feature.get(ATTRIBUTE_NAME)
        return code.getInfo()

    except Exception:
        return 'GEE 오류'

results_list = []
start_index = 0

if os.path.exists(OUTPUT_CSV_PATH):
    try:
        df_existing = pd.read_excel(OUTPUT_CSV_PATH)
        results_list = df_existing.to_dict('records')
        start_index = len(results_list)
    except Exception:
        results_list = []
        start_index = 0

df = pd.read_csv(INPUT_CSV_PATH)

total_rows = len(df)

start_time = time.time()

for index, row in df.iterrows():
    if index < start_index:
        continue

    fire_year = row[YEAR_COLUMN_NAME]
    latitude = row[LAT_COLUMN_NAME]
    longitude = row[LON_COLUMN_NAME]

    code = get_forest_info_for_year(fire_year, longitude, latitude)

    results_list.append({
        'lat': latitude,
        'lon': longitude,
        'forest': code
    })

    current_row_num = index + 1
    if (
        current_row_num > start_index
        and current_row_num % SAVE_INTERVAL == 0
        and current_row_num < total_rows
    ):
        temp_result_df = pd.DataFrame(results_list)
        temp_result_df.to_csv(OUTPUT_CSV_PATH, index=False)

end_time = time.time()

result_df = pd.DataFrame(results_list)
result_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
